# Development Program : #1

Print("Hello Fards!")


# Challenge: Space Challenges

Reference Link: https://github.com/rfordatascience/tidytuesday/tree/master/data/2019/2019-01-15


1) Initial Data Setup
  + The Soviet Union was renamed to Russia partway through the dataset.In the state_code column change all occurrences of "SU" to "RU"
  +  Exclude agency_type "private", "startup" from the dataset
  +  Add a column called launch_month that extracts the month from the launch_date column
  
2) Who are the top 4 countries (state_code) in space launch count?
  +  Create a data frame that shows each country, their respective count of launches, and their rank
  
3) For the top two countries (state_code) what is their success rate by decade
  +  Note: The category column marks whether a launch is successful. A launch is a success if category is "O" and unsuccessful if category is "F".
  + Success rate by decade is defined as the percentage of successful launches over all the launches performed by a particular country in a decade.

In [1]:
import pandas as pd 
launches_df = (pd.read_csv('https://raw.githubusercontent.com/rfordatascience/' + \
                           'tidytuesday/master/data/2019/2019-01-15/launches.csv')
              )

launches_df.head()
launches_df.shape

(5726, 11)

1) Initial Data Setup
  + The Soviet Union was renamed to Russia partway through the dataset.In the state_code column change all occurrences of "SU" to "RU"
 

In [2]:
launches_df['state_code'] = launches_df['state_code'].replace('SU', 'RU')

launches_df[launches_df['state_code'] == 'SU']

,tag,JD,launch_date,launch_year,type,variant,mission,agency,state_code,category,agency_type


 +  Exclude agency_type "private", "startup" from the dataset
  

In [3]:
launches_df[~launches_df.agency_type.isin(['private', 'startup'])].head()

,tag,JD,launch_date,launch_year,type,variant,mission,agency,state_code,category,agency_type
0,1967-065,2439671.38,1967-06-29,1967,Thor Burner 2,NaN,Secor Type II S/N 10,US,US,O,state
1,1967-080,2439725.70,1967-08-23,1967,Thor Burner 2,NaN,DAPP 3419,US,US,O,state
2,1967-096,2439774.83,1967-10-11,1967,Thor Burner 2,NaN,DAPP 4417,US,US,O,state
3,1968-042,2439999.69,1968-05-23,1968,Thor Burner 2,NaN,DAPP 5420,US,US,O,state
4,1968-092,2440152.69,1968-10-23,1968,Thor Burner 2,NaN,DAPP 6422,US,US,O,state


+  Add a column called launch_month that extracts the month from the launch_date column

In [4]:
launches_df['launch_date'] = pd.to_datetime(launches_df['launch_date'], errors='coerce')

launches_df['launch_month'] = launches_df['launch_date'].dt.month.astype("Int64")

launches_df.head()


,tag,JD,launch_date,launch_year,type,variant,mission,agency,state_code,category,agency_type,launch_month
0,1967-065,2439671.38,1967-06-29,1967,Thor Burner 2,NaN,Secor Type II S/N 10,US,US,O,state,6
1,1967-080,2439725.70,1967-08-23,1967,Thor Burner 2,NaN,DAPP 3419,US,US,O,state,8
2,1967-096,2439774.83,1967-10-11,1967,Thor Burner 2,NaN,DAPP 4417,US,US,O,state,10
3,1968-042,2439999.69,1968-05-23,1968,Thor Burner 2,NaN,DAPP 5420,US,US,O,state,5
4,1968-092,2440152.69,1968-10-23,1968,Thor Burner 2,NaN,DAPP 6422,US,US,O,state,10


2) Who are the top 4 countries (state_code) in space launch count?
  +  Create a data frame that shows each country, their respective count of launches, and their rank

In [5]:
launches_counts_df = launches_df['state_code'].value_counts().rename_axis('state_code').reset_index(name='counts')

launches_counts_df

,state_code,counts
0,RU,3178
1,US,1716
2,CN,302
3,F,291
4,J,115
5,IN,65
6,I-ESA,13
7,IL,10
8,I,9
9,IR,8


In [6]:
launches_counts_df['rank'] = launches_counts_df['counts'].rank(ascending = False).astype(int)

launches_counts_df.head(4)

,state_code,counts,rank
0,RU,3178,1
1,US,1716,2
2,CN,302,3
3,F,291,4


3) For the top two countries (state_code) what is their success rate by decade
  +  Note: The category column marks whether a launch is successful. A launch is a success if category is "O" and unsuccessful if category is "F".

In [7]:
launches_df['decade'] = launches_df['launch_year'] // 10 *10

In [8]:
success_count_df = launches_df[launches_df['state_code'].isin(['RU', 'US'])].groupby(['state_code','decade','category'])['tag']\
.count().to_frame('counts').reset_index()

success_count_df.head(10)

,state_code,decade,category,counts
0,RU,1950,F,5
1,RU,1950,O,6
2,RU,1960,F,58
3,RU,1960,O,378
4,RU,1970,F,38
5,RU,1970,O,866
6,RU,1980,F,22
7,RU,1980,O,931
8,RU,1990,F,16
9,RU,1990,O,411


 + Success rate by decade is defined as the percentage of successful launches over all the launches performed by a particular country in a decade.

In [9]:
success_wide_df = success_count_df.pivot_table(index = ['state_code', 'decade'], columns = 'category', values = 'counts')\
.reset_index().rename_axis(None, axis=1)

success_wide_df['success_rate'] = success_wide_df['F']/(success_wide_df['F'] + success_wide_df['O'])

success_wide_df[['state_code', 'decade', 'success_rate']]

,state_code,decade,success_rate
0,RU,1950,0.454545
1,RU,1960,0.133028
2,RU,1970,0.042035
3,RU,1980,0.023085
4,RU,1990,0.037471
5,RU,2000,0.025105
6,RU,2010,0.043269
7,US,1950,0.560976
8,US,1960,0.119626
9,US,1970,0.051282
